# Database Load

In [2]:
# Dependencies and Setup
import json
import os
import pandas as pd
import urllib.request
import requests
# from config import db_pwd, db_user
from config import URI
from sqlalchemy import create_engine

## Load Data into DataFrame

In [3]:
csv_file = "data/Merged_data.csv"
google_data_df = pd.read_csv(csv_file)
google_data_df.head()

,Unnamed: 0,states,dates,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential,SMA_retail_recreation,SMA_grocery_pharmacy,SMA_parks,SMA_transit,SMA_workplaces,SMA_residential,case_count,new_case_count,revenue_all,revenue_ss60,deaths
0,0,Alabama,2020-02-15,5.155172,0.230769,49.571429,5.761905,0.784314,-0.863636,-4.353333,3.583871,1.666667,4.946667,-22.844444,7.441667,0.0,0.0,0.0647,0.00421,NaN
1,1,Alabama,2020-02-16,-1.454545,-4.015625,-5.476190,2.136364,-1.920000,0.523810,-4.335632,3.376344,1.800000,5.013333,-21.935461,7.108333,0.0,0.0,0.0427,-0.00607,NaN
2,2,Alabama,2020-02-17,-1.169492,-4.476190,12.933333,5.727273,-16.264706,4.333333,-4.324138,3.105556,2.000000,5.140000,-21.568085,6.916667,0.0,0.0,-0.0220,-0.07060,NaN
3,3,Alabama,2020-02-18,-4.280702,-5.000000,-12.466667,-0.681818,0.338235,1.435897,-4.388506,2.746667,0.966667,4.946667,-20.662411,6.641667,0.0,0.0,0.0392,-0.02160,NaN
4,4,Alabama,2020-02-19,1.438596,-0.730159,6.666667,3.714286,1.397059,0.000000,-4.066667,2.704444,1.466667,4.973333,-19.695035,6.291667,0.0,0.0,0.0347,-0.01220,NaN


In [4]:
google_df = google_data_df[["states","dates","SMA_retail_recreation", "SMA_grocery_pharmacy", "SMA_parks",
                            "SMA_transit", "SMA_workplaces", "SMA_residential", "case_count", "new_case_count", "revenue_all", "revenue_ss60", "deaths"]]
google_df.head()

,states,dates,SMA_retail_recreation,SMA_grocery_pharmacy,SMA_parks,SMA_transit,SMA_workplaces,SMA_residential,case_count,new_case_count,revenue_all,revenue_ss60,deaths
0,Alabama,2020-02-15,-4.353333,3.583871,1.666667,4.946667,-22.844444,7.441667,0.0,0.0,0.0647,0.00421,NaN
1,Alabama,2020-02-16,-4.335632,3.376344,1.800000,5.013333,-21.935461,7.108333,0.0,0.0,0.0427,-0.00607,NaN
2,Alabama,2020-02-17,-4.324138,3.105556,2.000000,5.140000,-21.568085,6.916667,0.0,0.0,-0.0220,-0.07060,NaN
3,Alabama,2020-02-18,-4.388506,2.746667,0.966667,4.946667,-20.662411,6.641667,0.0,0.0,0.0392,-0.02160,NaN
4,Alabama,2020-02-19,-4.066667,2.704444,1.466667,4.973333,-19.695035,6.291667,0.0,0.0,0.0347,-0.01220,NaN


In [5]:
google_df = google_df.rename(columns = {"SMA_retail_recreation":'sma_retail_recreation',
                            "SMA_grocery_pharmacy":'sma_grocery_pharmacy', "SMA_parks":'sma_parks',"SMA_transit":'sma_transit',"SMA_workplaces":'sma_workplaces',"SMA_residential":'sma_residential'})
google_df

,states,dates,sma_retail_recreation,sma_grocery_pharmacy,sma_parks,sma_transit,sma_workplaces,sma_residential,case_count,new_case_count,revenue_all,revenue_ss60,deaths
0,Alabama,2020-02-15,-4.353333,3.583871,1.666667,4.946667,-22.844444,7.441667,0.0,0.0,0.0647,0.00421,NaN
1,Alabama,2020-02-16,-4.335632,3.376344,1.800000,5.013333,-21.935461,7.108333,0.0,0.0,0.0427,-0.00607,NaN
2,Alabama,2020-02-17,-4.324138,3.105556,2.000000,5.140000,-21.568085,6.916667,0.0,0.0,-0.0220,-0.07060,NaN
3,Alabama,2020-02-18,-4.388506,2.746667,0.966667,4.946667,-20.662411,6.641667,0.0,0.0,0.0392,-0.02160,NaN
4,Alabama,2020-02-19,-4.066667,2.704444,1.466667,4.973333,-19.695035,6.291667,0.0,0.0,0.0347,-0.01220,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8767,Wyoming,2020-07-31,19.620000,18.425000,NaN,35.429167,-22.344444,3.244444,2726.0,45.9,-0.2040,-0.05510,26.0
8768,Wyoming,2020-08-01,18.860000,17.600000,NaN,35.379167,-21.597778,3.122222,2769.0,46.1,-0.2090,-0.05220,26.0
8769,Wyoming,2020-08-02,19.056667,17.066667,NaN,34.629167,-20.858974,2.788889,2808.0,47.6,-0.2210,-0.11100,26.0
8770,Wyoming,2020-08-03,20.236667,17.200000,NaN,34.991667,-21.266667,2.977778,2848.0,46.9,-0.2070,-0.09200,27.0


In [5]:
# Lisa Connection to the database
rds_connection_string = f"{db_user}:{{db_pwd}}@localhost:5432/mobility_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [6]:
# Stojancho Connection to the database
# rds_connection_string = f"postgres:{pcode}@localhost:5432/mobility2_db"
engine = create_engine(f'{URI}')

### Check for tables

In [7]:
engine.table_names()

['merged_data']

### Use pandas to load csv converted DataFrame into database

In [8]:
google_df.to_sql(name='merged_data', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the tables

In [9]:
pd.read_sql_query('select * from merged_data', con=engine).head(10)

,id,states,dates,sma_retail_recreation,sma_grocery_pharmacy,sma_parks,sma_transit,sma_workplaces,sma_residential,case_count,new_case_count,revenue_all,revenue_ss60,deaths
0,1,Alabama,2020-02-15,-4.353333,3.583871,1.666667,4.946667,-22.844444,7.441667,0.0,0.0,0.0647,0.00421,NaN
1,2,Alabama,2020-02-16,-4.335632,3.376344,1.800000,5.013333,-21.935461,7.108333,0.0,0.0,0.0427,-0.00607,NaN
2,3,Alabama,2020-02-17,-4.324138,3.105556,2.000000,5.140000,-21.568085,6.916667,0.0,0.0,-0.0220,-0.07060,NaN
3,4,Alabama,2020-02-18,-4.388506,2.746667,0.966667,4.946667,-20.662411,6.641667,0.0,0.0,0.0392,-0.02160,NaN
4,5,Alabama,2020-02-19,-4.066667,2.704444,1.466667,4.973333,-19.695035,6.291667,0.0,0.0,0.0347,-0.01220,NaN
5,6,Alabama,2020-02-20,-4.083908,2.260000,0.366667,4.373333,-19.302128,6.300000,0.0,0.0,0.0430,-0.02750,NaN
6,7,Alabama,2020-02-21,-3.660920,2.240000,0.533333,4.720000,-18.565957,6.116667,0.0,0.0,0.0403,-0.06630,NaN
7,8,Alabama,2020-02-22,-3.370115,2.260000,3.266667,4.980000,-17.406383,5.766667,0.0,0.0,0.0223,-0.07770,NaN
8,9,Alabama,2020-02-23,-3.227586,2.212644,2.466667,5.426667,-16.481560,5.433333,0.0,0.0,0.0201,-0.08140,NaN
9,10,Alabama,2020-02-24,-3.319540,1.827778,2.233333,5.360000,-15.587234,5.225000,0.0,0.0,-0.0635,-0.12200,NaN


In [10]:
pd.read_sql_query('select sma_workplaces from merged_data', con=engine).head(10)

,sma_workplaces
0,-22.844444
1,-21.935461
2,-21.568085
3,-20.662411
4,-19.695035
5,-19.302128
6,-18.565957
7,-17.406383
8,-16.481560
9,-15.587234
